Preparation of the World Ocean Database data for the period and depth of interest.     
The domain is defined in the variable `dom` (tuple).

In [ ]:
using NCDatasets
using PhysOcean
using DIVAnd
using PyPlot
using Dates
using Statistics
const plt = PyPlot
using PyCall
using TOML
mpl = PyCall.pyimport("matplotlib")
mpl.style.use("calanus.mplstyle")
include("../src/InterpCalanus.jl")
usecartopy = false

In [ ]:
config = TOML.parsefile("./config.ini")
dom = config["domain"]

In [ ]:
if usecartopy
    
    ccrs = PyCall.pyimport("cartopy.crs")
    cfeature = PyCall.pyimport("cartopy.feature")
    coast = cfeature.GSHHSFeature(scale="full");
    cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
    mainproj = ccrs.Mercator(central_longitude=0.5*(dom[1] + dom[2]),
        min_latitude=dom[3], max_latitude=dom[4])
    datacrs = ccrs.PlateCarree();
end

In [ ]:
datadir = "/media/ctroupin/My Passport/data/WOD/NorthSea-EMODnetBio"
outputdir = "../data/WOD"
figdir = config["directories"]["wodfigdir"]
varname = "Temperature"
mycolor = "#6667AB"
isdir(outputdir) ? @debug("ok") : mkpath(outputdir)
isdir(figdir) ? @debug("ok") : mkpath(figdir);

In [ ]:
outputfile = joinpath(outputdir, "temperature_surface_WOD2.nc")

if isfile(outputfile)
    @info("Already read the data from WOD");
    @time obsvalue, obslon, obslat, obsdepth, obstime, obsids = loadobs(Float64, outputfile, varname);
    
else
    @info("Reading data from WOD and re-writing in a new netCDF");
    
    #@time obsvalue, obslon, obslat, obsdepth, obstime, obsid = 
    #WorldOceanDatabase.load(Float64, datadir, varname);

    #@time DIVAnd.saveobs(outputfile, 
    #        varname, obsvalue[gooddepth], 
    #        (obslon[gooddepth], obslat[gooddepth], obsdepth[gooddepth], obstime[gooddepth]), 
    #        obsid[gooddepth]);
end

### Select near-surface observations

In [ ]:
gooddepth = findall(obsdepth .< 5.);
@info(length(gooddepth));

## Time histogram
Get the years and months for the observations near surface (i.e., shallower than 5 m).

### Monthly histogram

In [ ]:
yearcount, monthcount = InterpCalanus.count_years_months(obstime[gooddepth]);

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.bar(1:12, monthcount, color=mycolor)
ax.set_xticks(collect(1:12))
ax.set_xticklabels(Dates.monthname.(1:12))
ax.set_xlim(0.5, 12.5)
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)
ax.spines["bottom"].set_visible(false)
fig.autofmt_xdate()
ax.set_title("Monthly distribution of observations from WOD")
plt.savefig(joinpath(figdir, "WOD_time_histogram_month"))
# plt.show()
plt.close()

### Month polar

In [ ]:
N = 12
theta = LinRange(0, 2 * π - π/6, N)
width = (1.8 * π) / N
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=true)
ax.set_thetagrids(collect(0:30:330), Dates.monthname.(1:12))
ax.set_theta_zero_location("N")
bars = ax.bar(theta, monthcount, width=width, color=mycolor)
plt.savefig(joinpath(figdir, "WOD_time_histogram_month_polar"))
plt.close()

### Year

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.bar(minimum(years):maximum(years), yearcount, color=mycolor)
ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)
ax.spines["bottom"].set_visible(false)
ax.set_title("Yearly distribution of observations from WOD")

plt.savefig(joinpath(figdir, "WOD_time_histogram_year"))
# plt.show()
plt.close()

## Spatial distribution

In [ ]:
fig = plt.figure()
ax = plt.subplot(111, projection=mainproj)
ax.set_extent(dom)
ax.plot(obslon, obslat, "ko", markersize=1, transform=datacrs)
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = false
gl.right_labels = false
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
gl.xlabel_style = Dict("size" => 10)
gl.ylabel_style = Dict("size" => 10)
ax.set_title("Locations of the observations")
plt.savefig(joinpath(figdir, "location_obs"))
plt.show()
#plt.close()